In [ ]:
# 引入所需的套件
# ObsPy 需要：
import obspy
from obspy import UTCDateTime
import numpy as np
import matplotlib.pyplot as plt

# PyGMT 需要：
import pygmt

# --------------------------------------------------
# Part 1: ObsPy 應用 - 模擬波形並繪製
# --------------------------------------------------
print("--- ObsPy 模擬波形處理 ---")

# (1) 設定參數並創建一個空的 numpy 陣列作為波形數據
sampling_rate = 50.0  # 採樣率 50 Hz
npts = 5000           # 總點數
delta = 1.0 / sampling_rate
time = np.arange(npts) * delta

# (2) 模擬訊號：一個 1 Hz 的正弦波加上隨機噪音
data = 50 * np.sin(2 * np.pi * 1.0 * time)
data += 5 * np.random.randn(npts)

# (3) 創建 ObsPy Trace 物件並賦予 stat (元數據)
tr = obspy.Trace(data=data)
tr.stats.sampling_rate = sampling_rate
tr.stats.starttime = UTCDateTime(2025, 1, 1, 0, 0, 0)
tr.stats.station = "AI_STA"
tr.stats.channel = "BHZ"

# (4) 處理：進行 2 Hz 低通濾波
tr.filter('lowpass', freq=2.0, corners=4, zerophase=True)
print(f"ObsPy Trace 資訊: {tr.stats}")

# (5) 繪製波形
print("正在繪製模擬波形圖...")
tr.plot(title=f"Station: {tr.stats.station} | Processed Waveform", 
        fig=plt.figure())
plt.show()

# --------------------------------------------------
# Part 2: PyGMT 應用 - 繪製台灣周邊基礎地圖
# --------------------------------------------------
print("\n--- PyGMT 基礎地圖繪製 ---")

# (1) 建立 PyGMT 圖形物件
fig = pygmt.Figure()

# (2) 設定地圖參數 (台灣周邊)
region = [115, 125, 20, 30] 
projection = "M12c"  # Mercator 投影, 寬度 12 cm

# (3) 繪製底圖 (basemap)
# 'a' 自動標註軸線, +t 設定地圖標題
fig.basemap(
    region=region, 
    projection=projection, 
    frame=['a', '+t"Taiwan Region Basemap (PyGMT Demo)"'] 
)

# (4) 繪製陸地和海洋，並加入海岸線
fig.coast(
    land="sienna",       # 陸地顏色
    water="skyblue",     # 海洋顏色
    shorelines=True,     # 繪製海岸線
    resolution="f",      # 最快/最低解析度
)

# (5) 標註一個點 (例如台北的大致位置)
fig.plot(
    x=121.56, y=25.03,  # 經緯度
    style="s0.5c",      # 's' 代表方形標記, 0.5c 為大小
    color="red",        # 標記顏色
    pen="1p,black",     # 標記邊框
    label="Taipei"
)

# (6) 顯示圖形
print("正在繪製 PyGMT 地圖...")
fig.show()